### Handle ETH Canopy Height dataset with Zampy
Demo notebook for developers.

Import packages and configure paths.

In [1]:
import numpy as np
from zampy.datasets import EthCanopyHeight
from zampy.datasets.dataset_protocol import TimeBounds, SpatialBounds
from pathlib import Path

work_dir = Path("Path2work_dir")
download_dir = Path(work_dir, "download")
ingest_dir = Path(work_dir, "ingest")
times = TimeBounds(np.datetime64("2020-01-01"), np.datetime64("2020-12-31"))
bbox_demo = SpatialBounds(54, 6, 51, 3)

Download dataset.

In [2]:
canopy_height_dataset = EthCanopyHeight()
canopy_height_dataset.download(
    download_dir=download_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["canopy-height"],
)

File 'ETH_GlobalCanopyHeight_10m_2020_N51E003_Map.tif' already exists, skipping...


True

Data ingestion to the unified format in `zampy`.

In [3]:
canopy_height_dataset.ingest(download_dir, ingest_dir)

File 'ETH_GlobalCanopyHeight_10m_2020_N51E003_Map.nc' already exists, skipping...


True

In [4]:
ds = canopy_height_dataset.load(
    ingest_dir=ingest_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["canopy-height"],
)

In [5]:
ds

<xarray.Dataset>
Dimensions:        (longitude: 36000, latitude: 36000, time: 1)
Coordinates:
  * longitude      (longitude) float64 3.0 3.0 3.0 3.0 3.0 ... 6.0 6.0 6.0 6.0
  * latitude       (latitude) float64 51.0 51.0 51.0 51.0 ... 54.0 54.0 54.0
  * time           (time) datetime64[ns] 2020-07-01
Data variables:
    canopy-height  (time, latitude, longitude) float32 dask.array<chunksize=(1, 6000, 6000), meta=np.ndarray>

In [6]:
convert_dir = Path(work_dir, "convert")

canopy_height_dataset.convert(
    ingest_dir=ingest_dir,
    convert_dir=convert_dir,
    convention="ALMA",
)

Start converting data to follow the 'ALMA' convention.
Start processing file `ETH_GlobalCanopyHeight_10m_2020_N51E003_Map.nc`.
Variable 'canopy-height' is not included in 'ALMA' convention.
All variables already follow the ALMA convention or not included in the ALMA convention.
No conversion operation was performed on 'ETH_GlobalCanopyHeight_10m_2020_N51E003_Map.nc'.


For testing purpose only. <br>
Since the canopy height dataset doesn't have variable included in ALMA convention, we just fake a dataset to trigger the conversion step.

In [7]:
import xarray as xr

ds_fake = xr.Dataset(
    {
        "Qle":(
               ("lat", "lon"),
               np.random.rand(4).reshape(2, 2) * 5,)
    },
        coords={"lat": [10, 20], "lon": [150, 160]},
)

ds_fake.Qle.attrs["units"] = "watt/centimeter**2"
ds_fake

<xarray.Dataset>
Dimensions:  (lat: 2, lon: 2)
Coordinates:
  * lat      (lat) int64 10 20
  * lon      (lon) int64 150 160
Data variables:
    Qle      (lat, lon) float64 4.752 0.5508 2.361 0.5542

In [8]:
# save the fake datasets to ingest directory (with a different file name)
ds_fake.to_netcdf(
    Path(
        ingest_dir,
        "eth-canopy-height",
        "ETH_GlobalCanopyHeight_10m_2020_N51E003_fake_Map.nc",
    )
)

In [9]:
# Let's run the convert code again
canopy_height_dataset.convert(
    ingest_dir=ingest_dir,
    convert_dir=convert_dir,
    convention="ALMA",
)

Start converting data to follow the 'ALMA' convention.
Start processing file `ETH_GlobalCanopyHeight_10m_2020_N51E003_fake_Map.nc`.
Conversion of dataset 'ETH_GlobalCanopyHeight_10m_2020_N51E003_fake_Map.nc' following ALMA convention is complete!
Start processing file `ETH_GlobalCanopyHeight_10m_2020_N51E003_Map.nc`.
Variable 'canopy-height' is not included in 'ALMA' convention.
All variables already follow the ALMA convention or not included in the ALMA convention.
No conversion operation was performed on 'ETH_GlobalCanopyHeight_10m_2020_N51E003_Map.nc'.


In [10]:
# load converted dataset
ds_convert = xr.load_dataset(
    Path(
        convert_dir,
        "eth-canopy-height",
        "ETH_GlobalCanopyHeight_10m_2020_N51E003_fake_Map.nc",
    )
)
ds_convert

<xarray.Dataset>
Dimensions:  (lat: 2, lon: 2)
Coordinates:
  * lat      (lat) int64 10 20
  * lon      (lon) int64 150 160
Data variables:
    Qle      (lat, lon) float64 4.752e+04 5.508e+03 2.361e+04 5.542e+03

In [11]:
# check the conversion
assert np.allclose(ds_convert["Qle"].values / 10000, ds_fake["Qle"].values)